# Recherche de solutions

CSI4506 Intelligence Artificielle  
Automne 2020  
Préparé par Joel Muteba, Julian Templeton et Caroline Barrière

Ce notebook couvre la définition d'un espace d'états et des algorithmes de recherches.  Le code ci-bas est largement inspiré des ressources fournies avec le livre Artificial Intelligence, Foundations of Computational Agents (https://artint.info/2e/online.html).  En particulier, les ressources correspondant au chapitre 3, sur le thème de *Searching for Solutions*, se trouvent ici https://artint.info/AIPython/.

Il existe même un livre de Python complémentaire (en anglais) qui founi des explications du code, ainsi qu’un premier chapitre intitulé Python pour AI où ils donnent quelques notions de base sur Python. Voir https://artint.info/AIPython/aipython.pdf

Le code ci-bas est inspiré du code dans le fichier *searchProblem.py*.  Vous n'avez pas besoin de retourner à ce code.  Tout ce dont vous avez besoin pour votre travail est dans le présent notebook.  J'ai modifié le code du livre, et je le présente ici, par petits bouts, pour permettre à tous (surtout ceux et celles moins familiers avec python) de parcourir le code petit à petit. 

***DEVOIR***:  
Parcourir le notebook, cellule par cellule.  
Lorsque vous voyez **(TO DO)**, faites les tâches demandées.  Lorsque vous avez terminé, soumettez votre notebook.
***

**1. Definition d'un graphe dirigé**  
Un graphe dirigé est construit à partir de noeuds et d'arcs.  La classe ci-bas permet de définir des arcs qui relient deux noeuds.  Les deux premiers paramètres donnent les deux noeuds.  Les deux autres paramètres donnnent le coût d'un arc (par défaut mis à 1) et une action d'un arc (par défaut à None).  La méthode *init* est le constructeur.  La méthode *assert* permet de valider que le coût d'un arc ne peut être négatif.

In [3]:
class Arc(object):
    """An arc has a from_node and a to_node node and a (non-negative) cost"""
    
    def __init__(self, from_node, to_node, cost=1, action=None):
        assert cost >= 0, ("Cost cannot be negative for"+
                           str(from_node)+"->"+str(to_node)+", cost: "+str(cost))
        self.from_node = from_node
        self.to_node = to_node
        self.action = action
        self.cost=cost

    def __repr__(self):
        """string representation of an arc"""
        if self.action:
            return str(self.from_node)+" --"+str(self.action)+"--> "+str(self.to_node)
        else:
            return str(self.from_node)+" --> "+str(self.to_node)

Voici un petit exemple de graphe dirigé acyclic (DAG -- Directed Acyclic Graph).
        <img src="Image 1.png" />
        

Nous pouvons construire un DAG avec la classe Arc ci-haut définie.

In [4]:
# Defining a small dag
dag1 = [Arc('a','b'), Arc('b','c1'), Arc('c1','d'), Arc('b','c2'), Arc('c2','d')]

In [5]:
# print the dag
print(dag1)

[a --> b, b --> c1, c1 --> d, b --> c2, c2 --> d]


Aucune classe ne définit un noeud.  Pour l'instant, nous supposons qu'un noeud est une chaîne de caractères (String).

**2. Espace de recherche (search space)**  
Plutôt que de définir un graphe par un ensemble d'arcs, nous pouvons définir plus formellement une classe représentant un espace de recherhe qui contiendra non seulement une liste d'arcs, mais aussi une liste explicite de noeuds, un noeud initial, et un noeud final.  La classe contient aussi des méthodes pour obtenir de l'information sur les voisins d'un noeud, sur l'état d'un noeud (noeud final ou non).  La classe permet aussi de définir une fonction pour les heuristiques, ce que nous utiliserons plus tard.

In [6]:
class Search_problem_from_explicit_graph():
    """A search problem consists of:
    * a list or set of nodes
    * a list or set of arcs
    * a start node
    * a list or set of goal nodes
    * a dictionary that maps each node into its heuristic value.
    """

    def __init__(self, nodes, arcs, start=None, goals=set(), hmap={}):
        self.neighs = {}
        self.nodes = nodes
        for node in nodes:
            self.neighs[node]=[]
        self.arcs = arcs
        for arc in arcs:
            self.neighs[arc.from_node].append(arc)
        self.start = start
        self.goals = goals
        self.hmap = hmap

    def start_node(self):
        """returns start node"""
        return self.start
    
    def is_goal(self,node):
        """is True if node is a goal"""
        return node in self.goals

    def neighbors(self,node):
        """returns the neighbors of node"""
        return self.neighs[node]

    def heuristic(self,node):
        """Gives the heuristic value of node n.
        Returns 0 if not overridden in the hmap."""
        if node in self.hmap:
            return self.hmap[node]
        else:
            return 0
        
    def __repr__(self):
        """returns a string representation of the search problem"""
        res=""
        for arc in self.arcs:
            res += str(arc)+".  "
        return res

    def neighbor_nodes(self,node):
        """returns an iterator over the neighbors of node"""
        return (path.to_node for path in self.neighs[node])


Ci-bas, un exemple d'un espace de recherche à représenter. Le problème vise à aller du noeud 'a' au noeud 'h' (en jaune ci-bas). La définition de l'espace est présentée en dessous de l'image.

<img src="Image 2.png"/>

In [7]:
# Defining a search space
problemSimple = Search_problem_from_explicit_graph(
    {'a','b','c','d','e', 'f', 'g', 'h', 'j'},
    [Arc('a','b',1), Arc('a','c',1), Arc('b','d',1), Arc('b','e',1),
     Arc('c','f',1), Arc('c', 'g'), Arc('d','h',1), Arc('e','h',1),
     Arc('f', 'e', 1), Arc('f','j',1)],
    start = 'a',
    goals = {'h'})

In [8]:
# Print some informations
print(problemSimple.neighbors('a'))

[a --> b, a --> c]


**(TO DO) Question 1 (2 points)**  
Afficher le noeud initial de l'espace *problemSimple*.  Tester si le noeud 'g' est un noeud final.  Passer au travers de tous les noeuds définis dans *problemSimple* pour trouver lequel est un noeud final.

In [9]:
# Print the start node. 
print("The start node is: " + problemSimple.start_node())
# 
# Test if node 'g' is a goal node. 
if problemSimple.is_goal('g') :
    print("The node g is a goal node")
else:
    print("The node g is NOT a goal node")
# 
# Go through the list of nodes to find which one is the goal.

for node in problemSimple.nodes:
    if problemSimple.is_goal(node):
        print("The goal node is: " + node)


The start node is: a
The node g is NOT a goal node
The goal node is: h


**(TO DO) Question 2 (1 point)**  
Compléter la définition de *problemTest* qui définit le graphe ci-bas. N'oubliez pas d'inscrire les coûts des arcs.

<img src="Image 3.png"/>

In [10]:
# Define Problem Test    
problemTest = Search_problem_from_explicit_graph(
    {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'j', 'k', 'm', 'n'},
    [Arc('a','b',1), Arc('a','c',2), Arc('b','d',3), Arc('b','e',4),
     Arc('c','f',1), Arc('c', 'g',1), Arc('d','h',1), Arc('e','h',2),
     Arc('f', 'e', 1), Arc('f','j',5), Arc('j','n',2), Arc('h','m',4), 
     Arc('h','k',1), Arc('m','n',3)],
    start = 'a',
    goals = {'n'}
)
print(problemTest)

a --> b.  a --> c.  b --> d.  b --> e.  c --> f.  c --> g.  d --> h.  e --> h.  f --> e.  f --> j.  j --> n.  h --> m.  h --> k.  m --> n.  


**3. Definition d'une solution comme un chemin.**  
Une solution à un problème de recherche est un chemin, qui montre les noeuds visités pour se rendre du noeud initial au noeud final.  La classe ci-bas sert à définir un chemin. Pour l'instant, il faut voir qu'un chemin peut être un noeud (dans le cas du noeud initial) ou un chemin (définition récursive) suivi d'un arc. Ne vous inquiétez pas des détails d'implémentation.  Lorsque vos connaissances de python seront plus approfondies, vous comprendrez mieux.

In [11]:
# class to represent a path
class Path(object):
    """A path is either a node or a path followed by an arc"""
    
    def __init__(self,initial,arc=None):
        """initial is either a node (in which case arc is None) or
        a path (in which case arc is an object of type Arc)"""
        self.initial = initial
        self.arc=arc
        if arc is None:
            self.cost=0
        else:
            self.cost = initial.cost+arc.cost

    def end(self):
        """returns the node at the end of the path"""
        if self.arc is None:
            return self.initial
        else:
            return self.arc.to_node

    def nodes(self):
        """enumerates the nodes for the path.
        This starts at the end and enumerates nodes in the path backwards."""
        current = self
        while current.arc is not None:
            yield current.arc.to_node
            current = current.initial
        yield current.initial

    def initial_nodes(self):
        """enumerates the nodes for the path before the end node.
        This starts at the end and enumerates nodes in the path backwards."""
        if self.arc is not None:
            for nd in self.initial.nodes(): yield nd     # could be "yield from"
        
    def __repr__(self):
        """returns a string representation of a path"""
        if self.arc is None:
            return str(self.initial)
        elif self.arc.action:
            return (str(self.initial)+"\n   --"+str(self.arc.action)
                    +"--> "+str(self.arc.to_node))
        else:
            return str(self.initial)+" --> "+str(self.arc.to_node)

**4.  Definition d'un algorithme de recherche générique**  
Nous définissons une classe abstraite qui représente un algorithme de recherche générique.  La stratégie de gestion de la frontière sera raffinée dans les diverses classes plus spécifiques.

In [12]:
class GenericSearcher():
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    This does depth-first search unless overridden
    """
    def __init__(self, problem):
        """creates a searcher from a problem
        """
        self.problem = problem
        self.initialize_frontier()

    def initialize_frontier(self):
        self.frontier = [Path(self.problem.start_node())]
        
    def empty_frontier(self):
        return self.frontier == []
       
    ### change the add_to_frontier method --- THIS METHOD NEEDS TO BE IMPLEMENTED FOR DIFFERENT SEARCH STRATEGIES
    def add_to_frontier(self,path):
        raise NotImplementedError
        
    def search(self):
        """returns (next) path from the problem's start node
        to a goal node. 
        Returns None if no path exists.
        """
        
        while not self.empty_frontier():
            # The next node explored is selected
            # It is implemented with a Pop, which actually removes from the end of the list
            path = self.frontier.pop()            
            print(path)
            if self.problem.is_goal(path.end()):    # solution found
                self.solution = path                # store the solution found
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                for arc in neighs: # Loop over neighbors arcs
                    self.add_to_frontier(Path(path,arc))              

**5. Implémentation des recherches aveugles.**  
En modifiant la méthode *add_to_frontier* de la recherche générique, nous pourrons implémenter les trois types de recherche aveugle vus en classe.  La recherche en profondeur, en largeur, et de moindre coût.  La recherche en largeur est implémentée ci-bas, et vous devrez implémenter les deux autres types de recherche aveugle. 

In [13]:
# (CB) extend the generic searcher to be a Breadth-first searcher.
class BreadthFirstSearcher(GenericSearcher):

    def __init__(self, problem):   
        super().__init__(problem)

    def add_to_frontier(self,path):
        # breadth
        self.frontier.insert(0,path)        # HERE, I insert in position 0, knowing that I want a FIFO

In [14]:
# Test breadth-first searcher
searcherBreadth = BreadthFirstSearcher(problemTest)  
print("Exploring nodes:")
foundPath = searcherBreadth.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> c
a --> b --> d
a --> b --> e
a --> c --> f
a --> c --> g
a --> b --> d --> h
a --> b --> e --> h
a --> c --> f --> e
a --> c --> f --> j
a --> b --> d --> h --> m
a --> b --> d --> h --> k
a --> b --> e --> h --> m
a --> b --> e --> h --> k
a --> c --> f --> e --> h
a --> c --> f --> j --> n
Path found: a --> c --> f --> j --> n with a cost of 10


**(TO DO) Question 3 (2 points)**  
Modifier la méthode *add_fo_frontier* pour obtenir une recherche en profondeur.  Tester votre implémentation. 

In [15]:
# Depth-first searcher
class DepthFirstSearcher(GenericSearcher):
    def __init__(self, problem):   
        super().__init__(problem)

    def add_to_frontier(self,path):
        # To implement the frontier of the in depth algorithm, 
        # we need to append the newest path at the end of the list 
        self.frontier.append(path)

In [16]:
# Test
searcherDepth = DepthFirstSearcher(problemTest)  
print("Exploring nodes:")
foundPath = searcherDepth.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> c
a --> c --> g
a --> c --> f
a --> c --> f --> j
a --> c --> f --> j --> n
Path found: a --> c --> f --> j --> n with a cost of 10


**(TO DO) Question 4 (5 points)**  
Modifier la méthode *add_fo_frontier* pour obtenir une recherche de moindre coût.  Tester votre implémentation. 

In [17]:
# Lowest Cost searcher
class LowCostSearcher(GenericSearcher):
    def __init__(self, problem):   
        super().__init__(problem)

    def add_to_frontier(self,path):
        
        #First, we insert the path at the beginning of the list, supposing it is the most costly path
        self.frontier.insert(0,path)
        
        # For every node in the frontier, we compare the node with the given path
        # If the path is lest costly, the path will take the place of the node in the list
        # That way, the minimum is always at the end of the list
        for node in self.frontier:
            if node != path: # We don't want to compare the same node in the list
                if path.cost < node.cost: # compare the entire path cost
                    path_index = self.frontier.index(path)
                    node_index = self.frontier.index(node)
                    #switch places
                    self.frontier[path_index], self.frontier[node_index] =  self.frontier[node_index],  self.frontier[path_index]
                    

  

In [18]:
# Test
searcherLowCost = LowCostSearcher(problemTest)  
print("Exploring nodes:")
foundPath = searcherLowCost.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> c
a --> c --> f
a --> c --> g
a --> b --> d
a --> c --> f --> e
a --> b --> e
a --> b --> d --> h
a --> c --> f --> e --> h
a --> b --> d --> h --> k
a --> b --> e --> h
a --> c --> f --> e --> h --> k
a --> c --> f --> j
a --> b --> e --> h --> k
a --> b --> d --> h --> m
a --> c --> f --> e --> h --> m
a --> c --> f --> j --> n
Path found: a --> c --> f --> j --> n with a cost of 10


**6. Recherche heuristique**  
Nous avons vu trois types de recherche heuristique en classe: locale (greedy), meilleur-chemin d'abord, et A*. Tout comme les recherches aveugles, nous pouvons les implémenter en modifiant la méthodes add_to_frontier de la recherche générique.

Dans ce notebook, pour obtenir l'heuristique d'un nœud, vous devez appeler la fonction self.problem.heuristic (...) où vous passez le nœud pour lequel vous voulez l'heuristique. Un exemple de son utilisation peut être trouvé dans l'implémentation 6(b) de la recherche avec l'algorithme A*.

Supposons que la fonction heuristique donne une distance optimiste de chaque noeud au noeud 'n'.  Ces distances sont encodées dans un dictionnaire appelé *hmap*, dans la définition ci-bas, qui est une extension au problemTest définit précédemment.

In [19]:
problemTestWithHeuristics = Search_problem_from_explicit_graph(
    {'a','b','c','d','e', 'f', 'g', 'h', 'j', 'k', 'm', 'n'},
    [Arc('a','b', 1), Arc('a','c', 2), Arc('b','d',3), Arc('b','e',4),
     Arc('c','f',1), Arc('c','g',1), Arc('d', 'h', 1), Arc('e','h',2), 
     Arc('f','e',1), Arc('f','j',5), Arc('h', 'k',1), Arc('h','m',4), 
     Arc('m','n',3), Arc('j','n',2)],
    start = 'a',
    goals = {'n'},
     hmap = {'a':15, 'b':3, 'c':5, 'd':6, 'e':3, 'f':1, 'g':2, 'h':4, 'j':9, 'k':3, 'm':2, 'n':0})

***6 (a). Recherche heuristique locale***    
Comme discuté dans le cours, il existe deux types d'algorithmes de recherche heuristique. Le premier type est celui des recherches heuristiques locales. L'algorithme de recherche heuristique locale discuté en classe est la recherche Greedy (Greedy Search), qui suppose une vision limitée d'un problème.

**(TO DO) Question 5 - 5 points** <br> 
Pour votre premier algorithme heuristique, implémentez et testez l'algorithme de recherche heuristique locale Greedy Search dans les cellules ci-dessous. Rappelez-vous que l’algorithme Greedy Search examine les coûts locaux. Ainsi, cet algorithme devra revenir en arrière s’il atteind une impasse. Cela signifie que si un chemin d’accès entraîne une impasse, l’algorithme de recherche doit poursuivre la recherche à partir du nœud précédent dans ce chemin.

In [20]:
# Greedy searcher
class GreedySearcher(GenericSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):   
        super().__init__(problem)
    
    def add_to_frontier(self,path):

        # First, insert the path at the beginning of the list, we suppose it's the most costly path for the algorithm to take
        self.frontier.insert(0, path)
        neighs_path = self.problem.neighbors(path.end()) # Define the path neighbors

        # For every node in the frontier, we compare the node with the given path
        # Here, the algorithm is looking forward -- which means it will take only the cost of the node forward in the diagram
        # This algorithm normally doesn't have a frontier -- so I used recursivity to implement it
        # Bascially, I call the add to frontier path for every neighbord of a path that is lest costly than a node
        # It will try to find a way to the end node -- if it is not a goal node, the algorithm will backtrack and start again
        for node in self.frontier: 
            if node != path:
                if path.arc.cost < node.arc.cost: # Compare arc cost (the cost of the following node)
                    # if the path has neightbors, it will call the add to frontier function until it arrives at an end node 
                    if neighs_path: 
                        for arc in neighs_path: # Loop over neighbors arcs to find a solution
                            self.add_to_frontier(Path(path,arc))  #call the add to frontier again
                        break; # get out of the loop when it finds a solution   
                        

                

In [21]:
# Test
searcherGreedy = GreedySearcher(problemTestWithHeuristics)  
print("Exploring nodes:")
foundPath = searcherGreedy.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> c
a --> b --> d
a --> b --> e
a --> c --> f
a --> c --> f --> e
a --> c --> f --> e --> h
a --> c --> f --> e --> h --> k
a --> c --> f --> e --> h --> m
a --> c --> f --> j
a --> c --> g
a --> b --> d --> h
a --> b --> d --> h --> k
a --> b --> d --> h --> m
a --> b --> d --> h --> m --> n
Path found: a --> b --> d --> h --> m --> n with a cost of 12


***6 (b). Recherche heuristique globale***    
Le deuxième type de recherche heuristique est la recherche heuristique globale. L'algorithme de recherche A* et l'algorithme de recherche Best-first sont tous deux des algorithmes de recherche heuristique globale.

Avant d'implémenter l'algorithme de recherche Best-first, vous trouverez ci-dessous l'algorithme de recherche A * à utiliser comme référence.

In [22]:
class AStarSearcher(GenericSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):   
        super().__init__(problem)

    def getCostKey(self, path):
        return (path.cost + self.problem.heuristic(path.end()))
    
    def add_to_frontier(self,path):
        """add path to the frontier with the appropriate cost"""
        # add path to the frontier 
        self.frontier.append(path)
        # sort frontier so that all the elements are ordered from most costly to least costly
        self.frontier.sort(key=self.getCostKey, reverse=True)

In [23]:
# Test
searcherAStar = AStarSearcher(problemTestWithHeuristics)  
print("Exploring nodes:")
foundPath = searcherAStar.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> c
a --> c --> f
a --> c --> g
a --> c --> f --> e
a --> b --> e
a --> c --> f --> e --> h
a --> c --> f --> e --> h --> k
a --> b --> d
a --> b --> d --> h
a --> b --> d --> h --> k
a --> b --> d --> h --> m
a --> b --> e --> h
a --> b --> e --> h --> k
a --> b --> d --> h --> m --> n
Path found: a --> b --> d --> h --> m --> n with a cost of 12


**(TO DO) Question 6 - 5 points** <br> 
En vous servant de l'algorithme de recherche A* implémenté ci-dessus, implémentez et testez l'algorithme best-first search.

In [24]:
# Best first searcher
class BestFirstSearcher(GenericSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):   
        super().__init__(problem)
        
    def getHeuristicKey(self, path): #this function returns the heuristic key to sort the list
        return (self.problem.heuristic(path.end()))
    
    def add_to_frontier(self,path):
        # append the path at the end of the list
        self.frontier.append(path)
        # sort frontier so that all the elements are ordered from most costly to least costly using the heuristic cost
        self.frontier.sort(key=self.getHeuristicKey, reverse=True)

In [25]:
# Test
searcherBestFirst = BestFirstSearcher(problemTestWithHeuristics)  
print("Exploring nodes:")
foundPath = searcherBestFirst.search();
print('Path found: {} with a cost of {}'.format(foundPath, foundPath.cost))

Exploring nodes:
a
a --> b
a --> b --> e
a --> b --> e --> h
a --> b --> e --> h --> m
a --> b --> e --> h --> m --> n
Path found: a --> b --> e --> h --> m --> n with a cost of 14


***SIGNATURE:***
My name is Dounia Mansouri.
I certify being the author of this assignment.